In [1]:
from methods.llm import get_async_vllm_endpoint
import os 

# Unlimited LLM endpoints
endpoint_id = "vllm-rimbwy29f75muq"
api_key = "rpa_EPOJED42G59S80Y6SKMCOI330EQU4JPPMKV2UD2W7j0uku"
get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key)

Could not load vllm class, check CUDA support and GPU RAM size


#### Temasek Foundation Dataset

In [13]:
# Node dataset should ideally be concise and pure application info included ...
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.load_data import prep_tf_node, custom_metric_map

tf_meta_prompt, test_cases = prep_tf_node(prompt_mode=True)

# Specific Metric required for TF dataset (Worth refactor the code a bit)
from methods.llm import get_groq_response
from methods.evolnode import EvolNode 

starter_code_str = """def generate_prompt(project_description):
    return (
        f"Given the grant application description: ### Description ### \\n{project_description}\\n ### End of Description ###, do evaluate the project and make a decision to reject it. Make sure the output is a json string in markdown like this {{\'decision\':<boolean>, \'comment\':\'<string>\'}} near the top of your response or people will die."
    )""" # add initial code string
    
node = EvolNode(tf_meta_prompt, starter_code_str, "Basic Prompt Function", get_response=get_endpoint_response, test_cases=test_cases[:5], custom_metric_map=custom_metric_map) # setting manual test cases

In [8]:
node_dict = {"reasoning": "Basic Prompt Function", "code": starter_code_str, "fitness": 0.0}
node.evolve("e2", feedback="reject all cases", parents=[node_dict], batch_size=20) # Evaluation on test case completely failing here .... 


     :: Query time: 3.93s


Processing LLM queries:   0%|          | 0/20 [00:00<?, ?it/s]